In [2]:
from platform import python_version
import bnlearn as bn
from utils import *
import matplotlib.pyplot as plt
import pickle
import classifiers
import integrate_vbll
import torch
from sklearn.metrics import zero_one_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score
%matplotlib inline
python_version()

vbll found


'3.10.9'

In [4]:
import sys, os


VBLL_PATH_UPTODATE = "/home/pyla/bayesian/vbll_uptodate/vbll"

sys.path.append(os.path.abspath(VBLL_PATH_UPTODATE))

import vbll

In [7]:
vbll.vbll.DiscClassification

AttributeError: module 'vbll' has no attribute 'vbll'

In [8]:
from vbll.layers.classification import DiscClassification


ModuleNotFoundError: No module named 'vbll.layers'

In [5]:
help(vbll)

Help on package vbll:

NAME
    vbll

PACKAGE CONTENTS
    script
    script_mc
    script_noise
    setup
    vbll (package)

FILE
    (built-in)




In [2]:
exp_name = 'scene'
classes = ['Beach','Sunset','FallFoliage','Field','Mountain','Urban']

input_features = ['Att' + str(i) for i in range(1,295)]

train_data, train_labels = load_scene(partition='Train')
train_labels = make_binary(train_labels, classes)

X_train = train_data[input_features].values
y_train = train_labels[classes].values

X_train = X_train[y_train.sum(axis=1) != 0]
y_train = y_train[y_train.sum(axis=1) != 0]


test_data, test_labels = load_scene(partition='Test')
test_labels = make_binary(test_labels, classes)

X_test = test_data[input_features].values
y_test = test_labels[classes].values

X_test = X_test[y_test.sum(axis=1) != 0]
y_test = y_test[y_test.sum(axis=1) != 0]


# --- Device configuration ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cpu


In [3]:
# --- Hyper-parameters ---
input_size = len(input_features)
hidden_size = 100
n_hidden = 1
num_classes = len(classes)
batch_size = 128
num_epochs = 100
num_runs = 2
learning_rate = 1e-3

reg_weight = 1./X_train.__len__()
param = 'diagonal'
return_ood = False
prior_scale = 1.0
noise_label = True

In [4]:
vbll_clf_dict = {}
for run in range(num_runs):

    vbll_clf_dict[run] = {}

        
    print('run')
    print(run)

    train_data, train_labels = load_scene(partition='Train')
    train_labels = make_binary(train_labels, classes)

    X_train = train_data[input_features].values
    y_train = train_labels[classes].values

    X_train = X_train[y_train.sum(axis=1) != 0]
    y_train = y_train[y_train.sum(axis=1) != 0]


    test_data, test_labels = load_scene(partition='Test')
    test_labels = make_binary(test_labels, classes)

    X_test = test_data[input_features].values
    y_test = test_labels[classes].values

    X_test = X_test[y_test.sum(axis=1) != 0]
    y_test = y_test[y_test.sum(axis=1) != 0]

    clf = integrate_vbll.VBLLClassifierClf(input_size, hidden_size, n_hidden, num_classes, batch_size, num_epochs, learning_rate, reg_weight, param, return_ood, prior_scale, noise_label, device)

    clf.fit(X_train, y_train)

    preds, preds_proba = clf.predict_with_proba(X_test)
    print('0-1 Accuracy')
    print(1. - zero_one_loss(y_test, preds))
    vbll_clf_dict[run]['01_loss'] = zero_one_loss(y_test, preds)
    vbll_clf_dict[run]['y_test'] = y_test
    vbll_clf_dict[run]['y_pred'] = preds
    vbll_clf_dict[run]['y_pred_proba'] = preds_proba


run
0


AttributeError: module 'vbll' has no attribute 'DiscClassification'

In [ ]:
f = open(exp_name + "_vbll_results" + ".pkl","wb")
pickle.dump(vbll_clf_dict,f)
f.close()
